In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import sys
module_path = os.path.abspath(os.path.join('../../../../../../AgentTorch/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from AgentTorch.helpers import discrete_sample, logical_and, logical_not
from AgentTorch.substep import SubstepTransition

/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: /Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so
  Reason: image not found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  

In [2]:
# global variables

num_agents = 5
num_steps = 10
s_var = 0
r_var = 4
quarantine_days = 2
device = 'cpu'

In [3]:
def diff_sample(sample_prob, size, device, hard=True):
    probs = sample_prob * torch.ones(size).to(device)
    probs = torch.vstack((probs, 1.0 - probs)).transpose(0, 1)
    sampled_output = torch.nn.functional.gumbel_softmax(probs.log(),
                                                        tau=0.1,
                                                        hard=hard)[:, 0]
    return sampled_output

In [28]:
def _end_quarantine(t, is_quarantined, quarantine_start_date):
    agents_quarantine_end_date = quarantine_start_date + quarantine_days
    agent_quarantine_ends =  (t>= agents_quarantine_end_date).long()

    END_QUARANTINE_VAR = -1

    is_quarantined[t] += agent_quarantine_ends*END_QUARANTINE_VAR
    new_quarantine_start_date = torch.clone(quarantine_start_date)
    new_quarantine_start_date = new_quarantine_start_date + agent_quarantine_ends*()

    is_quarantined[t] = is_quarantined[t]*(1 - agent_quarantine_ends) + agent_quarantine_ends*(1 - is_quarantined[t])
    quarantine_start_date = quarantine_start_date*(1 - agent_quarantine_ends) + (num_steps + 1)*agent_quarantine_ends

    return is_quarantined, quarantine_start_date

def _start_quarantine(t, is_quarantined, exposed_infected_agents, quarantine_start_date, quarantine_start_prob):
    agents_quarantine_start = diff_sample(quarantine_start_prob,size=num_agents, device=device)
    agents_quarantine_start = logical_and(agents_quarantine_start, exposed_infected_agents)
    agents_quarantine_start = logical_and(logical_not(is_quarantined[t]), agents_quarantine_start)
    print("quarantine start: ", agents_quarantine_start)

    is_quarantined[t] = is_quarantined[t]*(1 - agents_quarantine_start) + agents_quarantine_start*(1 - is_quarantined[t])
    quarantine_start_date = quarantine_start_date*(1 - agents_quarantine_start) + (agents_quarantine_start)*t

    return is_quarantined, quarantine_start_date

def _break_quarantine(t, is_quarantined, quarantine_start_date, quarantine_break_prob):
    agents_quarantine_break = diff_sample(quarantine_break_prob, size=num_agents, device=device)
    agents_quarantine_break = logical_and(is_quarantined[t], agents_quarantine_break)

    is_quarantined[t] = is_quarantined[t]*(1 - agents_quarantine_break) + agents_quarantine_break*(1 - is_quarantined[t])
    quarantine_start_date = quarantine_start_date*(1 - agents_quarantine_break) + (num_steps + 1)*agents_quarantine_break

    return is_quarantined, quarantine_start_date

In [29]:
def update_quarantine_status(t, is_quarantined, infected_agents, quarantine_start_date, quarantine_start_prob, quarantine_break_prob):
    is_quarantined, quarantine_start_date = _end_quarantine(t, is_quarantined, quarantine_start_date)
    is_quarantined, quarantine_start_date = _start_quarantine(t, is_quarantined, infected_agents, quarantine_start_date, quarantine_start_prob)
    is_quarantined, quarantine_start_date = _break_quarantine(t, is_quarantined, quarantine_start_date, quarantine_break_prob)

    return is_quarantined, quarantine_start_date

In [30]:
t = 5
is_quarantined = torch.zeros((num_steps + 1, num_agents), dtype=torch.float32)
is_quarantined[3:6] = torch.tensor([1.0, 0.0, 1.0, 0.0, 1.0])

is_quarantined = torch.tensor(is_quarantined)

quarantine_start_date = torch.tensor([3, 11, 3, 11, 4])
disease_stage = torch.tensor([3, 1, 2, 0, 2])

quarantine_start_prob = torch.tensor(0.3, requires_grad=True)
quarantine_break_prob = torch.tensor(0.5, requires_grad=True)

not_susceptible = (disease_stage > s_var).long()
not_recovered = (disease_stage < r_var).long()

infected_agents = logical_and(not_susceptible, not_recovered)
print(infected_agents)

print(is_quarantined)

tensor([1, 1, 1, 0, 1])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


/var/folders/77/1x1h16z52dz3jbg4ddwv01pr0000gn/T/ipykernel_87419/3626438721.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  is_quarantined = torch.tensor(is_quarantined)


In [31]:
is_quarantined, quarantine_start_date  = _end_quarantine(t, is_quarantined, quarantine_start_date)

print(is_quarantined, quarantine_start_date)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]) tensor([11, 11, 11, 11,  4])


In [32]:
print(infected_agents)

is_quarantined, quarantine_start_date = _start_quarantine(t, is_quarantined, infected_agents, quarantine_start_date, quarantine_start_prob)

print(is_quarantined, quarantine_start_date)

tensor([1, 1, 1, 0, 1])
quarantine start:  tensor([0., 1., 0., 0., 0.], grad_fn=<AddBackward0>)
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], grad_fn=<CopySlices>) tensor([11.,  5., 11., 11.,  4.], grad_fn=<AddBackward0>)


In [33]:
is_quarantined, quarantine_start_date = _break_quarantine(t, is_quarantined, quarantine_start_date, quarantine_break_prob)

print(is_quarantined, quarantine_start_date)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]], grad_fn=<CopySlices>) tensor([11.,  5., 11., 11., 11.], grad_fn=<AddBackward0>)


In [64]:
# check for gradient flow
t = 5

is_quarantined = torch.zeros((num_agents), dtype=torch.float32)
is_quarantined[0] = 1
is_quarantined[2] = 1
is_quarantined[4] = 1

is_quarantined = torch.tensor(is_quarantined)

quarantine_start_date = torch.tensor([3, 11, 3, 11, 4])
disease_stage = torch.tensor([3, 1, 2, 0, 2])

quarantine_start_prob = torch.tensor(0.3, requires_grad=True)
quarantine_break_prob = torch.tensor(0.5, requires_grad=True)

not_susceptible = (disease_stage > s_var).long()
not_recovered = (disease_stage < r_var).long()
infected_agents = logical_and(not_susceptible, not_recovered)

def _new_start_quarantine(t, is_quarantined, exposed_infected_agents, quarantine_start_date, quarantine_start_prob):
    agents_quarantine_start = diff_sample(quarantine_start_prob,size=num_agents, device=device)
    agents_quarantine_start = logical_and(agents_quarantine_start, exposed_infected_agents)
    agents_quarantine_start = logical_and(logical_not(is_quarantined), agents_quarantine_start)

    print(agents_quarantine_start.requires_grad_)

    print("quarantine start: ", agents_quarantine_start)
    START_QUARANTINE_VAR = 1

    is_quarantined += agents_quarantine_start*START_QUARANTINE_VAR
    quarantine_start_date = quarantine_start_date*(1 - agents_quarantine_start) + (agents_quarantine_start)*t

    return is_quarantined, quarantine_start_date

def _new_end_quarantine(t, is_quarantined, quarantine_start_date):
    agents_quarantine_end_date = quarantine_start_date + quarantine_days
    agent_quarantine_ends =  (t>= agents_quarantine_end_date).long()

    END_QUARANTINE_VAR = -1

    is_quarantined += agent_quarantine_ends*END_QUARANTINE_VAR
    quarantine_start_date = quarantine_start_date*(1 - agent_quarantine_ends) + (num_steps + 1)*agent_quarantine_ends

    return is_quarantined, quarantine_start_date

is_quarantined, quarantine_start_date = _new_end_quarantine(t, is_quarantined, quarantine_start_date)
is_quarantined, quarantine_start_date = _new_start_quarantine(t, is_quarantined, infected_agents, quarantine_start_date, quarantine_start_prob)
print(is_quarantined)

is_quarantined.sum().backward()
print(quarantine_start_prob.grad)

<built-in method requires_grad_ of Tensor object at 0x157ed0a90>
quarantine start:  tensor([0., 1., 0., 0., 0.], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 1.], grad_fn=<AddBackward0>)
tensor(3.5047)


/var/folders/77/1x1h16z52dz3jbg4ddwv01pr0000gn/T/ipykernel_87419/1493024088.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  is_quarantined = torch.tensor(is_quarantined)


In [55]:
print(is_quarantined.shape)
print("------------------")

one_hot_t = F.one_hot(torch.tensor([t]), num_classes=num_steps+1).long()

print(torch.matmul(one_hot_t, is_quarantined.long()))


torch.Size([11, 5])
------------------
tensor([[0, 1, 0, 0, 0]])
